In [15]:
import pandas as pd
from EDAesp import *
from tqdm import tqdm
import random
import nltk
nltk.download('wordnet')
nltk.download('omw')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /home/hu/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw to /home/hu/nltk_data...
[nltk_data]   Package omw is already up-to-date!
[nltk_data] Downloading package stopwords to /home/hu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [16]:
from nltk.corpus import wordnet
synonyms = set()
word = 'shine'
if wordnet.synsets(word):
    for syn in wordnet.synsets(word):
        for l in syn.lemma_names('spa'):
            synonym = l.replace('_', " ").lower()
            synonym = "".join([char for char in synonym if (char.isalpha() or char == ' ')])
            synonyms.add(synonym)
if word in synonyms:
    synonyms.remove(word)
print(synonyms)

{'lucir', 'resplandor', 'reflejar', 'brillar', 'radiancia espectral', 'relucir', 'resplandecer', 'brillo', 'esplendor', 'relumbrar'}


In [17]:
eda(' contraria a anular el título de un estudiante de un centro privado que obtuvo el módulo de Técnico Superior en Higiene Bucodental sin acudir a claseEl Consejo ')

['contraria a anular el título de un estudiante de un centro privado que obtuvo el módulo de Técnico Superior en Higiene Bucodental sin acudir a claseEl Consejo ',
 'elevation contraria a anular el título de un estudiante de un centro privado que obtuvo el módulo de elevated railroad Técnico Superior en Higiene Bucodental sin acudir a claseEl Consejo',
 'contraria a anular el título de un estudiante de un centro privado que obtuvo el módulo de Técnico Superior en Higiene Bucodental sin acudir a claseEl Consejo',
 'contraria a anular el título de un estudiante de un centro privado que obtuvo el módulo de Técnico superscript en Higiene Bucodental sin acudir a claseEl Consejo',
 'contraria a anular el título de un estudiante de un centro privado que obtuvo el módulo de Técnico Superior en Higiene Bucodental sin united nations diamond state acudir a claseEl Consejo']

In [18]:
df = pd.read_json('data/elmoundo_1124.json')

In [19]:
df.head(10)

,category,content
0,internacional,Rusia cierra su embajada ante la OTAN y echará...
1,internacional,Rusia ha decidido suspender la labor de su emb...
2,internacional,"""En respuesta a las acciones de la OTAN, suspe..."
3,internacional,"""Si los miembros de la OTAN tienen algún asunt..."
4,internacional,Este 'portazo' de los rusos tendrá efectos fue...
5,internacional,Corea del Norte lanza un misil balístico desde...
6,internacional,Corea del Norte disparó al mar un misil balíst...
7,internacional,Así avanza la vacunación contra el Covid-19 en...
8,internacional,"A día de hoy, más de 3.749 millones de habita..."
9,internacional,El mundo está embarcado en un proceso de vacun...


In [20]:
content = df['content']
category = df['category']

In [21]:
category_dic = {}
for idx,cate in enumerate(list(set(df['category']))):
    category_dic[cate] = idx

In [22]:
category_dic

{'cultura': 0,
 'internacional': 1,
 'tecnologia': 2,
 'deportes': 3,
 'economia': 4,
 'ciencia-y-salud': 5,
 'television': 6}

In [23]:
total = 0
maxl,minl = 0,100000
dic_count = []
for ca in category_dic.keys():
    cur_len = len([i for i in category if i==ca])
    print('{}:{}'.format(ca,cur_len))
    maxl,minl = max(maxl,cur_len),min(minl,cur_len)
    dic_count.append(cur_len)
    total += cur_len
print('total:',total)
print('max_cate:{}\nmin_cate:{}'.format(maxl,minl))
print(dic_count)

cultura:7652
internacional:898
tecnologia:992
deportes:7681
economia:7682
ciencia-y-salud:1137
television:2024
total: 28066
max_cate:7682
min_cate:898
[7652, 898, 992, 7681, 7682, 1137, 2024]


#### find a proper increase num_aug

In [24]:
for ca in category_dic.keys():
    p = 4/(dic_count[category_dic[ca]]/(sum(dic_count)/len(dic_count)))
    print(category_dic[ca],dic_count[category_dic[ca]],p)

0 7652 2.0958852960943917
1 898 17.859370028635063
2 992 16.167050691244242
3 7681 2.0879721762419328
4 7682 2.0877003756462233
5 1137 14.105289609247395
6 2024 7.923771880293619


In [25]:
augumented_news = [[],[]]

In [26]:
for cate,cont in tqdm(zip(category,content)):
    if len(cont)>30:
        p = 4/(dic_count[category_dic[ca]]/(sum(dic_count)/len(dic_count)))
        new_conts = eda(cont,num_aug=max(6,int(p)))
        for new_cont in new_conts:
            augumented_news[0].append(cate)
            augumented_news[1].append(new_cont)
    else:
        augumented_news[0].append(cate)
        augumented_news[1].append(cont)

28066it [04:17, 109.20it/s]


In [27]:
len(augumented_news[0])

214806

In [28]:
total = 0
maxl,minl = 0,100000
dic_count = []
for ca in category_dic.keys():
    cur_len = len([i for i in augumented_news[0] if i==ca])
    print('{}:{}'.format(ca,cur_len))
    maxl,minl = max(maxl,cur_len),min(minl,cur_len)
    dic_count.append(cur_len)
    total += cur_len
print('total:',total)
print('max_cate:{}\nmin_cate:{}'.format(maxl,minl))
print(dic_count)

cultura:58597
internacional:6910
tecnologia:7633
deportes:59022
economia:58516
ciencia-y-salud:8685
television:15443
total: 214806
max_cate:59022
min_cate:6910
[58597, 6910, 7633, 59022, 58516, 8685, 15443]


In [29]:
shuffle_idx = [i for i in range(0,len(augumented_news[0]))]
random.shuffle(shuffle_idx)
shuffled_data = [[],[],[]]
for idx in shuffle_idx:
    shuffled_data[0].append(augumented_news[0][idx])
    shuffled_data[1].append(augumented_news[1][idx])    

In [30]:
augmented_df = {
    'category':shuffled_data[0],
    'content':shuffled_data[1]
}

In [31]:
augmented_df = pd.DataFrame(data=augmented_df)

In [32]:
augmented_df['category'].head(10)

0      economia
1      economia
2      deportes
3      deportes
4       cultura
5      deportes
6    tecnologia
7      economia
8      deportes
9      economia
Name: category, dtype: object

In [33]:
augmented_df.to_json('data/augmented_elmundo_1124.json')